In [1]:
import os
from simtk import unit
import foldamers
import cg_openmm
from simtk.openmm.app.pdbfile import PDBFile
from foldamers.cg_model.cgmodel import CGModel
from cg_openmm.simulation.tools import run_simulation
import numpy as np

/home/cvogel/software/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/cvogel/software/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# Job settings
output_directory = 'output'
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

In [3]:
print_frequency = 500  # Number of steps to skip when printing output
total_simulation_time = 50.0 * unit.nanosecond  # Units = picoseconds
simulation_time_step = 5.0 * unit.femtosecond
total_steps = round(total_simulation_time.__div__(simulation_time_step))
temperature = 300.0 * unit.kelvin

In [4]:
polymer_length = 12
backbone_lengths = [1]
sidechain_lengths = [1]
sidechain_positions = [0]
include_bond_forces = False
include_bond_angle_forces = True
include_nonbonded_forces = True
include_torsion_forces = True
constrain_bonds = True

In [5]:
# Bond definitions
bond_length = 7.5 * unit.angstrom
bond_lengths = {
    'bb_bb_bond_length': bond_length,
    'bb_sc_bond_length': bond_length,
    'sc_sc_bond_length': bond_length}
bond_force_constant = 0 * unit.kilocalorie_per_mole / unit.nanometer / unit.nanometer
bond_force_constants = {
    'bb_bb_bond_k': bond_force_constant,
    'bb_sc_bond_k': bond_force_constant,
    'sc_sc_bond_k': bond_force_constant}

In [6]:
mass = 100.0 * unit.amu
masses = {'backbone_bead_masses': mass, 'sidechain_bead_masses': mass}
r_min = 3.0 * bond_length  # Lennard-Jones potential r_min
# Factor of /(2.0**(1/6)) is applied to convert r_min to sigma
sigma = r_min / (2.0**(1 / 6))
sigmas = {'bb_sigma': sigma, 'sc_sigma': sigma}
epsilon = 0.5 * unit.kilocalorie_per_mole
epsilons = {'bb_eps': epsilon, 'sc_eps': epsilon}

In [7]:
# Bond angle definitions
bond_angle_force_constant = 0.5 * unit.kilocalorie_per_mole / unit.radian / unit.radian
bond_angle_force_constants = {
    'bb_bb_bb_angle_k': bond_angle_force_constant,
    'bb_bb_sc_angle_k': bond_angle_force_constant}
# OpenMM requires angle definitions in units of radians
bb_bb_bb_equil_bond_angle = 120.0 * (np.math.pi / 180.0)
bb_bb_sc_equil_bond_angle = 120.0 * (np.math.pi / 180.0)
equil_bond_angles = {
    'bb_bb_bb_angle_0': bb_bb_bb_equil_bond_angle,
    'bb_bb_sc_angle_0': bb_bb_sc_equil_bond_angle}

In [8]:
torsion_force_constant = 0.5 * unit.kilocalorie_per_mole / unit.radian / unit.radian
torsion_force_constants = {'bb_bb_bb_bb_torsion_k': torsion_force_constant}
# OpenMM requires angle definitions in units of radians
bb_bb_bb_bb_equil_torsion_angle = 78.0 * (np.math.pi / 180.0)
bb_bb_bb_sc_equil_torsion_angle = 78.0 * (np.math.pi / 180.0)
equil_torsion_angles = {
    'bb_bb_bb_bb_torsion_0': bb_bb_bb_bb_equil_torsion_angle}
torsion_periodicities = {'bb_bb_bb_bb_period': 1}

In [ ]:
positions = PDBFile("helix.pdb").getPositions()

In [ ]:
cgmodel = CGModel(
    polymer_length=polymer_length,
    backbone_lengths=backbone_lengths,
    sidechain_lengths=sidechain_lengths,
    sidechain_positions=sidechain_positions,
    masses=masses,
    sigmas=sigmas,
    epsilons=epsilons,
    bond_lengths=bond_lengths,
    bond_force_constants=bond_force_constants,
    bond_angle_force_constants=bond_angle_force_constants,
    torsion_force_constants=torsion_force_constants,
    equil_bond_angles=equil_bond_angles,
    equil_torsion_angles=equil_torsion_angles,
    torsion_periodicities=torsion_periodicities,
    include_nonbonded_forces=include_nonbonded_forces,
    include_bond_forces=include_bond_forces,
    include_bond_angle_forces=include_bond_angle_forces,
    include_torsion_forces=include_torsion_forces,
    constrain_bonds=constrain_bonds,
    positions=positions)

In [ ]:
energy_kn = np.zeros((len(configurations),replica_energies.shape[2]))
for i in range(len(replica_energies)):
    energy_kn = replica_energies[:,i,:]

In [ ]:
print("Running a simulation.")
run_simulation(
    cgmodel,
    output_directory,
    total_simulation_time,
    simulation_time_step,
    temperature,
    print_frequency)

exit()

In [ ]:
energy_bin = np.zeros((len(temperature_list),energy_kn.shape[1]))
figure = pyplot.figure(0)
for T_index in range(len(energy_kn)):
    num_samples = int(len(configurations[T_index]))
    pyplot.hist(energy_kn[T_index],num_samples,alpha=0.3)
    #pyplot.hist(energy_kn[T_index][:num_samples],energy_bin)
    
pyplot.xlabel("Replica Energy")
pyplot.ylabel("Count")
pyplot.title("Replica Energy vs. Count")
pyplot.savefig("energy_dist.png")
pyplot.legend([round(temperature._value,1) for temperature in temperature_list],loc='center left', bbox_to_anchor=(1, 0.5),title='T (K)')    
